# Negative Sampling for unbalance classes
+ 多値分類を二値分類に近似する
+ 多値分類時に使用するSoftmax計算の計算コストを低減する

In [9]:
import numpy as np

# Embedding
+ 単語の分散表現を保持＆抽出

``` 単語の分散表現(W,D=3)
forward   W
       |◯◯◯|
       |◯◯◯|
idx -> |◯◯◯| -> |◯◯◯|
       |◯◯◯|
       |◯◯◯|
backward  dW
       |◯◯◯|
       |◯◯◯| <- |●●●| idx
       |◯◯◯|
       |◯◯◯|
       |◯◯◯|
```

In [3]:
class Embedding:
    def __init__(self, w: np.ndarray):
        self.params = [w]
        self.grads = [np.zeros_like(w)]
        self.idx = None
    
    def forward(self, idx: int):
        w, = self.params
        self.idx = idx
        out = w[self.idx]
        return out
    
    def backward(self, dout):
        dw = self.grads
        dw[...] = 0 # dwの中身をすべて0. torch.zero_grad()と同じ
        for i, word_id in enumerate(self.idx):
            dw[word_id]+= dout[i]
        return None

## EmbeddingDot
+ 単語の分散表現を抽出して内積

In [4]:
class EmbeddingDot:
    def __init__(self, w: np.ndarray):
        self.embed = Embedding(w)
        self.params = self.embed
        self.grads = self.embed.grads
        self.cache = None
    
    def forward(self, h: np.ndarray, idx) -> np.ndarray:
        target_w: np.ndarray = self.embed.forward(idx)
        out = np.sum(target_w * h, axis=1) # 内積 -> スコア
        self.cache = (h, target_w)
        return out
    
    def backward(self, dout: np.ndarray):
        h, target_w = self.cache
        dout = dout.reshape(dout.shape[0], 1)
        d_target_w = dout * h
        self.embed.backward(d_target_w)
        dh = dout
        return dh

# UnigramSampler
+ コーパスから＊＊に基づいてサンプリング

In [13]:
import collections

class UnigramSampler:
    def __init__(self, corpus: list, power: float, sample_size: int):
        self.sample_size = sample_size
        self.vocab_size = None # 単語数
        self.word_p = None # 確率分布
        # 単語の出現回数を取得
        counts = collections.Counter()
        for word_id in corpus:
            counts[word_id] += 1 # 昇順(0,1,2,...)に頻度が格納される
        self.vocab_size = len(counts)
        self.word_p = np.zeros(self.vocab_size)
        # 確率分布(ターゲットを含む)
        self.word_p = np.power(self.word_p, power)
        self.word_p /= np.sum(self.word_p)

    def get_negative_sample(self, target):
        batch_size = target.shape[0]
        negative_sample = np.zeros((batch_size, self.sample_size), dtype=np.int32)
        for i in range(batch_size):
            p = np.copy(self.word_p)
            target_idx = target[i]
            p[target_idx] = 0 # ターゲット(正例)を省いた確率分布を作成
            p /= np.sum(p)
            # サンプリング
            negative_sample = np.random.choice(p, 
                                               size=(batch_size, self.sample_size),
                                               replace=False, # 重複なし
                                               p=p)
        return negative_sample

# SigmoidWithLoss

In [16]:
class SigmoidWithLoss:
    def __init__(self):
        self.params = []
        self.grads = []
        self.loss = None
        self.y = None # sigmoidの出力
        self.t = None # 教師label

    def forward(self, x: np.ndarray, t: np.ndarray):
        self.t = t # (N, 1)
        self.y = 1 / (1 + np.exp(-x))
        # cross entroy loss
        self.loss = -self.t * np.log(self.y) + (1 - self.t) * np.log(1 - self.y)

        return self.loss
    
    def backward(self, dout = 1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) * dout / batch_size
        return dx

In [17]:
import numpy as np


class NegativeSamplingLoss:
    def __init__(self, w: np.ndarray, corpus: list, power = 0.75, sample_size=5):
        self.sample_size = sample_size # 負例数1
        self.sampler = UnigramSampler(corpus, power, sample_size)
        self.loss_layers = [SigmoidWithLoss() for _ in range(sample_size + 1)]
        self.embed_dot_layers = [EmbeddingDot(w) for _ in range(sample_size + 1)]
        self.params, self.grads = [], []

        for layer in self.embed_dot_layers:
            self.params += layer.params
            self.grads += layer.grads
        
    def forward(self, h: np.ndarray, target: np.ndarray):
        batch_size = target.shape[0]
        # ネガティブサンプリング
        negative_samples = self.sampler.get_negative_sample(target)

        # 正例(0)
        score = self.embed_dot_layers[0].forward(h, target)
        correct_list = np.ones(batch_size, dtype=np.int32) # すべて1のベクトル
        loss = self.loss_layers[0].forward(score, correct_list)

        # 負例(1)
        negative_label = np.zeros(batch_size, dtype=np.int32)
        for i in range(self.sample_size):
            negative_target = negative_samples[:, i]
            score = self.embed_dot_layers[i + 1].forward(h, negative_target)
            loss += self.loss_layers[i + 1].forward(score, negative_label)
        return loss
    
    def backward(self, dout=1):
        dh = 0 # 準伝搬でRepeatしていた各コピー
        for l0, l1 in zip(self.loss_layers, self.embed_dot_layers):
            dscore = l0.backward(dout)
            dh += l1.backward(dscore)
        return dh    

## EmbeddingDotレイヤ
```                                        
          ----------------                 ---------
say(1) -> | EmbeddingDot |    Yes/No (1) ->|       |            
          | Layer        | --------------->| Sigmod| -> Loss
h(1,D) -> |------------- |      t - y      | with  |
          |    W_out     |                 | Loss  |
          ----------------                 ---------
```

```
h ------------             ----------------                 ---------
    |      | 正例say(1) -> | EmbeddingDot |    Yes/No (1) ->|       |            
    |      |               | Layer        | --------------->| Sigmod| -> Loss (+)
    |      |---->h(1,D) -> |------------- |      t - y      | with  |
    |                      |    W_out     |                 | Loss  |
    |                      ----------------                 ---------
    |
    |                      ----------------                 ---------
    |     負例 hello(0) -> | EmbeddingDot |    Yes/No (1) ->|       |            
    |                      | Layer        | --------------->| Sigmod| -> Loss (+)
    |----------->h(1,D) -> |------------- |      t - y      | with  |
    |                      |    W_out     |                 | Loss  |
    |                      ----------------                 ---------
    ・・・・
```

## Negative Samplingのサンプリング手法
＋コーパスの確率分布に従ってサンプリング
+ e.g. p = [0.5, 0.1, 0.05, 0.2, 0.05, 0.1] (sum=1)
+ `np.random.choice(words, p=p, size=2)`

### 確率の低い単語のサンプリングを救済する方法
+ すべての確率に0.75乗する
```
P'(w_i) = P(w_i)^0.75 / sum(P(w_i)^0.75)
```